In [2]:
import numpy as np
from protocol import *

# Testing the calibration

In [ ]:
msg = "This is some random message that actually isn't all that long"
bin_msg = generate_fake_message(msg, ratio=4.5)

In [ ]:
ratio, hist = calibrate_protocol(bin_msg)

In [7]:
print('The real ratio is: {}, the calibrated ratio is: {}'.format(4.5, ratio))

The real ratio is: 4.5, the calibrated ratio is: 4.484848484848484


# Testing the protocol

In [45]:
def next_byte(ratio, min_len, testing_iter=None):
    while True:
        ignored_bits = ''
        # Wait until a '\x00' char to start reading data
        # Wait until a 0 bit is read
        next_bit = '1'
        while next_bit == '1':
            if testing_iter is None:
                next_bit = str(rgpio.read(15)) # TODO: Change this line
            else:
                try:
                    next_bit = str(next(testing_iter))
                except StopIteration:
                    return
            ignored_bits += str(next_bit)

        # Wait till length of current_bits is long enough for 7 real bits
        current_bits = '0'
        while True:
            if testing_iter is None:
                next_bit = str(rgpio.read(15)) # TODO: Change this line
            else:
                try:
                    next_bit = str(next(testing_iter))
                except StopIteration:
                    return
            ignored_bits += str(next_bit)
            current_bits += str(next_bit)
            if next_bit == 1:
                current_bits = ''
            if len(current_bits) > int(7*ratio) + 1:
                break

        if testing_iter is not None:
            print('Ignored bits are:', ignored_bits)
        # Start reading next byte
        while True:
            if testing_iter is not None:
                print('Getting next byte:')
                print('Bits are: \'', end='')
            byte = ''
            current_bits = '0'
            # Read rest of bits
            count = 0
            while True:
                # Read bit (either till switch or min_length + 1 times)
                if testing_iter is None:
                    next_bit = str(rgpio.read(15)) # TODO: Change this line
                else:
                    try:
                        next_bit = str(next(testing_iter))
                    except StopIteration:
                        return
                    print(next_bit, end='')

                if next_bit != current_bits[-1]: # Switch
                    # Get the number of bits written 
                    num_bits = int(round(len(current_bits) / ratio))
                    for i in range(num_bits):
                        byte += current_bits[-1]
                    current_bits = ''
                current_bits += next_bit

                if len(byte) >= 6:
                    break


            if testing_iter is not None:
                print('\nRead bits are:', byte)
            byte = chr(int(byte, 2))
            if testing_iter is not None:
                print('Ascii is:', byte)
            yield byte

In [46]:
# Create a message to encode and decode
msg = 'This is the test message'

binary_msg = '0000000'+'0000000'+'0000000'+'0000000'+'0000' + generate_fake_message(msg)
print(binary_msg)

Char T, ascii 84, bin 0b1010100
Char h, ascii 104, bin 0b1101000
Char i, ascii 105, bin 0b1101001
Char s, ascii 115, bin 0b1110011
Char  , ascii 32, bin 0b100000
Char i, ascii 105, bin 0b1101001
Char s, ascii 115, bin 0b1110011
Char  , ascii 32, bin 0b100000
Char t, ascii 116, bin 0b1110100
Char h, ascii 104, bin 0b1101000
Char e, ascii 101, bin 0b1100101
Char  , ascii 32, bin 0b100000
Char t, ascii 116, bin 0b1110100
Char e, ascii 101, bin 0b1100101
Char s, ascii 115, bin 0b1110011
Char t, ascii 116, bin 0b1110100
Char  , ascii 32, bin 0b100000
Char m, ascii 109, bin 0b1101101
Char e, ascii 101, bin 0b1100101
Char s, ascii 115, bin 0b1110011
Char s, ascii 115, bin 0b1110011
Char a, ascii 97, bin 0b1100001
Char g, ascii 103, bin 0b1100111
Char e, ascii 101, bin 0b1100101
00000000000000000000000000000000111100000111100000111100000000011111111100000111100000000000000111111111000011111000000000111111111111111111000000000111111111111100000000000000000000000111111111000011111000000000111111

In [50]:
def msg_iter(msg=binary_msg):
    for bit in msg:
        yield bit
testing_iter = msg_iter()

In [51]:
re_created_msg = ''
for _ in range(18):   
    try:
        re_created_msg += next(next_byte(4.5, 4, testing_iter=testing_iter))
    except StopIteration:
        print('')
        
print(re_created_msg)

Ignored bits are: 000000000000000000000000000000001
Getting next byte:
Bits are: '1110000011110000011110000000001
Read bits are: 1010100
Ascii is: T
Ignored bits are: 11111111000001111000000000000001111111110
Getting next byte:
Bits are: '000111110000000001111111111111111110
Read bits are: 01001111
Ascii is: O
Ignored bits are: 000000001111111111111000000000000
Getting next byte:
Bits are: '0000000000011111111100001
Read bits are: 000110
Ascii is: 
Ignored bits are: 1111000000000111111111111111111000000
Getting next byte:
Bits are: '0001111111111111000000000000000000000001
Read bits are: 011100000
Ascii is: à
Ignored bits are: 111111111111000001111000000000111111111000001
Getting next byte:
Bits are: '111000000000000001111111110
Read bits are: 100011
Ascii is: #
Ignored bits are: 000000001111000001111111110000000
Getting next byte:
Bits are: '000000000000000111111111111110
Read bits are: 0000111
Ascii is: 
Ignored bits are: 000111110000000001111111110000000
Getting next byte:
Bits ar